 Importing Librares

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import cv2
import tensorflow as tf
import keras
import os
from tqdm import tqdm
from keras.preprocessing.image import img_to_array
import pandas as pd
from tensorflow.keras.layers import BatchNormalization
import random
import cv2  
import imutils
from skimage import color
from skimage import io


# Data Preprocessing

## X_Data

In [ ]:
SIZE= 250
Path1='/content/drive/MyDrive/διπλωματικη/data/my data/images'
Path2='/content/drive/MyDrive/διπλωματικη/data/my data/moires/file.csv'
files=os.listdir(Path1)
train_y = []
eikones=[]


In [ ]:
for i in tqdm(range(1,3398)):
  i=str(i)
  image=cv2.imread(Path1+'/'+'image-'+i+'.png',0)
  image = imutils.rotate(image, angle= 180)
  img=cv2.resize(image,(SIZE, SIZE))
  train_y.append(img_to_array(img))
  eikones.append(image)

In [ ]:
image_array = np.reshape(train_y, (len(train_y), SIZE, SIZE, 1))
image_array = image_array.astype('float32') / 255.
train_array= image_array[:3000,:,:]
test_array=image_array[3001:3300,:,:]
val_array=image_array[3301:,:,:]

### Show my images

In [ ]:
figure =plt.figure(dpi=1200)
plt.subplot(231)
plt.title('Sample 1')
plt.imshow(eikones[0], cmap='gray',origin="lower")
plt.axis("off")
plt.subplot(232)
plt.title('Sample 2')
plt.imshow(eikones[1], cmap='gray',origin="lower")
plt.axis("off")
plt.subplot(233)
plt.title('Sample 3')
plt.imshow(eikones[3001], cmap='gray',origin="lower")
plt.axis("off")
plt.subplot(234)
plt.title('Sample 4')
plt.imshow(eikones[3002], cmap='gray',origin="lower")
plt.axis("off")
plt.show()

## Y_Data

In [ ]:
dataset = pd.read_csv(Path2)
y = dataset.iloc[:, -1].values
y=-y


In [ ]:
for i in range(0,3398):
  print(y[i])

In [ ]:
train_y_array= y[:3000]
test_y_array=y[3001:3300]
val_y_array=y[3301:]



# Model

In [ ]:
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
import numpy
from keras.layers import Input
from keras.layers import UpSampling2D
from keras.layers.convolutional import Conv2D, Conv2DTranspose 
from keras.layers.pooling import MaxPooling2D
from keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger,ReduceLROnPlateau
import pickle

In [ ]:
from keras.layers.core import activation
from tensorflow.keras.layers import BatchNormalization
SIZE=250
input= Input(shape=(SIZE,SIZE,1))
c= Conv2D(256,(3,3),strides=(1,1),padding='same',activation='relu')(input)
c=BatchNormalization()(c)
c=MaxPooling2D(pool_size=(2,2), strides=(2,2))(c)
c= Conv2D(128,(3,3),strides=(1,1),padding='same',activation='relu')(c)
c=BatchNormalization()(c)
c=MaxPooling2D(pool_size=(2,2), strides=(2,2))(c)
c= Conv2D(64,(3,3),strides=(1,1),padding='same',activation='relu')(c)
c=BatchNormalization()(c)
c=MaxPooling2D(pool_size=(2,2), strides=(2,2))(c)
c=Conv2D(filters=64, kernel_size=(3, 3),activation='relu', padding='same')(c)
c= tf.keras.layers.GlobalMaxPooling2D()(c)
c=tf.keras.layers.Flatten()(c)
c=tf.keras.layers.Dense(512, activation='relu')(c)
c=tf.keras.layers.Dropout(0.1)(c)
c=tf.keras.layers.Dense(256, activation='relu')(c)
c=tf.keras.layers.Dropout(0.2)(c)
c=tf.keras.layers.Dense(64, activation='relu')(c)
c=tf.keras.layers.Dropout(0.2)(c)
output=tf.keras.layers.Dense(1, activation=tf.keras.activations.linear)(c)
myfriend= Model(inputs=input,outputs=output)
myfriend.summary()



# Fit My Model

In [ ]:
optimizer = tf.keras.optimizers.Adam(lr=0.001)
myfriend.compile(optimizer = optimizer,loss = 'mse')
callbacks= [EarlyStopping(monitor='loss',patience=10,verbose=1,restore_best_weights=True),CSVLogger('/content/drive/MyDrive/διπλωματικη/models/Logmyfriend.csv'),ReduceLROnPlateau(monitor="val_loss",factor=0.5,patience=10,verbose=1,min_lr=0.0002)]
results=myfriend.fit(train_array,train_y_array,batch_size=24,epochs=100,callbacks=callbacks,validation_split=0.1)

# Feature visualization 1

In [ ]:
layer_names = [layer.name for layer in myfriend.layers]
layer_names

In [ ]:
layer_outputs = [layer.output for layer in myfriend.layers]
layer_outputs

In [ ]:
feature_map_model = tf.keras.models.Model(inputs=myfriend.input, outputs=layer_outputs)

In [ ]:
image_path='/content/drive/MyDrive/διπλωματικη/data/chech.png'
image=cv2.imread(image_path,0)
image=cv2.resize(image,(250, 250))
input=img_to_array(image)
input = np.reshape(input, (1, SIZE, SIZE, 1))
input = input.astype('float32') / 255.0


In [ ]:
feature_maps = feature_map_model.predict(input)

In [ ]:
for layer_name, feature_map in zip(layer_names, feature_maps):
  print(f"The shape of the {layer_name} is =======>> {feature_map.shape}")

In [ ]:
for layer_name, feature_map in zip(layer_names, feature_maps):  
  if len(feature_map.shape) == 4:
      k = feature_map.shape[-1]  
      size=feature_map.shape[1]
      image_belt = np.zeros((size, size*k),np.uint8)
      for i in range(k):
        feature_image = feature_map[0, :, :, i]
        feature_image-= feature_image.mean()
        feature_image/= feature_image.std ()
        feature_image*=  64
        feature_image+= 128
        feature_image= np.clip(feature_image, 0, 255).astype('uint8')
        image_belt[:, i * size : (i + 1) * size] = feature_image
        scale = 20. / k
        plt.figure( figsize=(scale * k, scale) )
        plt.title ( layer_name )
        plt.grid  ( False )
        plt.imshow( image_belt,cmap='gray')
      #columns = size
      #rows = size
      #for ftr in feature_map:
        #fig=plt.figure(figsize=(12, 12))
        #for i in range(1,k):
          #fig =plt.subplot(rows, columns, i)
          #fig.set_xticks([])  #Turn off axis
          #fig.set_yticks([])
          #plt.imshow(ftr[ :, :, i-1], cmap='gray')
          
        #plt.show()

# Prediction

In [ ]:
test_img_input=np.expand_dims(test_array[2], 0)
pred = myfriend.predict(test_img_input)
#angle=sc.inverse_transform(pred)[0][0]
#print(angle)
print('original degrees:',test_y_array[2])
#angle=round(pred,0)
print('predicted degrees:',pred)
  
 

## Average Error Deviation

### Known Data

In [ ]:
from sklearn.preprocessing import StandardScaler
import math
MAE=0
#scaler = StandardScaler()
for i in tqdm(range(0,3000)):
  test_img_input=np.expand_dims(train_array[i], 0)
  pred = myfriend.predict(test_img_input)
#angle=sca.inverse_transform(pred)[0][0]
  #print(pred)
#angle=round(pred,0)
#print(prend)
  
  U=train_y_array[i]-pred
  tet=abs(U)
  
  MAE=MAE+tet

print(MAE)

In [ ]:
print(MAE/3000)

### Unknown Data

In [ ]:
from sklearn.preprocessing import StandardScaler
import math
MSE_t=0
#scaler = StandardScaler()
for i in tqdm(range(0,299)):
  test_img_input=np.expand_dims(test_array[i], 0)
  pred = myfriend.predict(test_img_input)
#angle=sca.inverse_transform(pred)[0][0]
  #print(pred)
#angle=round(pred,0)
#print(prend)
  
  U_t=test_y_array[i]-pred
  tet=abs(U_t)
  
  MSE_t=MSE_t+tet
print(MSE_t)

In [ ]:
print(MSE_t/299)

##TOP 80

### Known Data

In [ ]:
known_data=[]

In [ ]:
from sklearn.preprocessing import StandardScaler
import math
MΑE=0
#scaler = StandardScaler()
for i in tqdm(range(0,3000)):
  test_img_input=np.expand_dims(train_array[i], 0)
  pred = myfriend.predict(test_img_input)
#angle=sca.inverse_transform(pred)[0][0]
  #print(pred)
#angle=round(pred,0)
#print(prend)
  
  U=train_y_array[i]-pred
  tet=abs(U)
  known_data.append(tet)

In [ ]:
print(known_data)

In [ ]:
known_data.sort()

In [ ]:
for i in range(0,3000):
  print(known_data[i])

In [ ]:
Top80=[]

In [ ]:
for i in range(0,2400):
  Top80.append(known_data[i])

In [ ]:
print(len(Top80))

In [ ]:
Top80_1=0
for i in range(0,2400):
  Top80_1=Top80_1 + Top80[i]

In [ ]:
print(Top80_1/2400)

In [ ]:
add=0

for i in tqdm(range(0,299)):
  test_img_input=np.expand_dims(test_array[i], 0)
  pred = myfriend.predict(test_img_input)
  U=abs(test_y_array[i]-pred)
  if U>5 :
    add=add+1
    
print(add)




In [ ]:
print(len(brisko))

In [ ]:
loss = results.history['loss']
val_loss = results.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()